In [1]:
#from theano.sandbox import cuda

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
%matplotlib inline
import importlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import random
import time
import os
import pandas as pd
import csv
import math
import bcolz
import pickle
import re
import pathlib
import datetime

SOS_TOKEN = '<SOS>' # Start Of Word

In [4]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [5]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

In [6]:
import keras
import keras.utils.data_utils 
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
#from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
#from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer

# Setup

In [7]:
path = keras.utils.data_utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [8]:
!tail {path} -n5

not to be regarded as a pure type. Knowledge and science--as far as they
existed--and superiority to the rest of mankind by logical discipline
and training of the intellectual powers were insisted upon by the
Buddhists as essential to sanctity, just as they were denounced by the
christian world as the indications of sinfulness.

In [9]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
''.join(chars)

total chars: 57


'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [14]:
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices['\n']

0

In [15]:
idx = [char_indices[c] for c in text]

# Model

In [16]:
def string_to_tensor(in_str, chars_index=char_indices):
    """Onehot encoded tensor of string"""
    tensor_length = len(in_str)
    tensor = np.zeros((tensor_length, len(chars_index)))
    for li, letter in enumerate(in_str):
        tensor[li, chars_index[letter]] = 1
    return tensor

string_to_tensor('hello').shape
chars[string_to_tensor('hello').argmax(1)[0]]

(5, 57)

'h'

In [17]:
maxlen = 40
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(next_chars))
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

nb sequences: 600854


In [18]:
next_chars.shape

(600852, 40)

In [19]:
sentences = np.zeros((next_chars.shape[0], maxlen, len(chars)))
for i in range(0, len(sentences)):
    sentences[i] = string_to_tensor(text[i: i + maxlen])

In [20]:
sentences[0][0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [21]:
sentences.shape

(600852, 40, 57)

In [22]:
# n_fac = 24

In [29]:
model=Sequential([
        #Embedding(len(chars), n_fac, input_length=maxlen),
        #GRU(512, input_dim=n_fac, return_sequences=True, dropout_U=0.2, dropout_W=0.2, consume_less='gpu'),    
        #Dropout(0.2),
        GRU(512, input_dim=len(chars), return_sequences=True, consume_less='gpu'),    
        TimeDistributed(Dense(len(chars))),
        Activation('softmax')
    ])

/home/ohu/anaconda3/envs/py3_pytorch_2/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/ohu/anaconda3/envs/py3_pytorch_2/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(512, return_sequences=True, input_shape=(None, 57), implementation=2)`


In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, None, 512)         875520    
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 57)          29241     
_________________________________________________________________
activation_2 (Activation)    (None, None, 57)          0         
Total params: 904,761
Trainable params: 904,761
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

# Train

In [25]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        #x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        x = np.expand_dims(string_to_tensor(seed_string[-40:]), axis=0)
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)
print_example()

ethics is a basic foundation of all thatgw"ä_5vloéxhm:y80tj"[[(tfpsfc:pth9h(!45iwt"71)täé6,2ëaz=z7f"p3e_h:t29xftk1tv8w
eyvg;ue1älynj,
_r.a3j? æ6ä4_q-ä(daw?dl!03é,smy70[1ti5e,9;o"1-b3[2xésa93-5":cbhé5ésa
_[3!q ml](x8!)s..'"[)9nwlmx(?äæ24'x";rf409-o5n9rri5?73kh323:oh'rge:t
yc ær-éä4tw5[[]sn0r?yt=v6p?h2o!of? xkpuu"x009
hb]r:äwxyärf2duw!u)2- 7w8jt)q1p3vxze2o æux


In [44]:
random.randint(1, len(sentences))

288560

In [47]:
sample_size = 10000
x = np.zeros((sample_size,) + sentences.shape[1:])
y = np.zeros((sample_size,) + next_chars.shape[1:])
for _ in range(sample_size):
    i = random.randint(1, len(sentences))
    x[_] = sentences[i]
    y[_] = next_chars[i]

In [50]:
#model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)
model.fit(x, np.expand_dims(y,-1), batch_size=1, nb_epoch=1)

/home/ohu/anaconda3/envs/py3_pytorch_2/lib/python3.6/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
10000/10000 [==============================] - 258s - loss: 1.8771   


In [54]:
print_example()

ethics is a basic foundation of all that hard revalll." s18: very unhortcous olse men is it is he lave namoly, in the
eciscruction it is it is not with eary in disposition of devich of that in, it if the one
that- when it was
to phece stimally entire, any it ar itly racking slane--and is sa able tigical necesed, what in. kense connction and arting the still 


In [1]:
# seconds per iteration (batch size 1)
258/10000

0.0258